In [59]:
import sympy as sp
import numpy as np
from IPython.display import display, Math

In [51]:
def transX(d):
    return sp.Matrix([[1, 0, 0, d],
                     [0, 1, 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
def transZ(a):
    return sp.Matrix([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, a],
                     [0, 0, 0, 1]])

def rotX(theta):
    return sp.Matrix([[1, 0, 0, 0],
                     [0, sp.cos(theta), -sp.sin(theta), 0],
                     [0, sp.sin(theta), sp.cos(theta), 0],
                     [0, 0, 0, 1]])

def rotZ(theta):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, 0],
                     [sp.sin(theta), sp.cos(theta), 0, 0],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    
def rotMattoRPY(R):
    return sp.Matrix([sp.atan2(R[1,0], R[0,0]), sp.atan2(-R[2,0], sp.sqrt(R[2,1]**2 + R[2,2]**2)), sp.atan2(R[2,1], R[2,2])])

In [73]:
l0, l1, l2 = sp.Symbol('l0'), sp.Symbol('l1'), sp.Symbol('l2')

DH = sp.Matrix([[0, 0, l0, -sp.pi/2],
               [0, -sp.pi/2, 0, -sp.pi/2],
               [0, sp.pi/2, l1, 0],
               [0, 0, l2, 0]])

#type of joints
jointType = sp.Matrix([1, 1, 1, 0])
# where 1 is revolute and 0 is prismatic

# homogeneous transformation matrix
Hne = sp.Matrix([[0, 0, 1, 0],
                [0, -1, 0, 0],
                [1, 0, 0, 0],
                [0, 0, 0, 1]])
display(Math(r'\mathbf{DH} = ' + sp.latex(DH)))
display(Math(r'\mathbf{Hne} = ' + sp.latex(Hne)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [74]:
Hiden = sp.eye(4)
q = [sp.Symbol('q1'), sp.Symbol('q2'), sp.Symbol('q3'), sp.Symbol('q4')]
for i in range(DH.shape[0]):
    Tx = transX(DH[i, 0])
    Rx = rotX(DH[i, 1])
    Tz = transZ(DH[i, 2])
    Rz = rotZ(DH[i, 3])
    if jointType[i]:
        Hj = rotZ(q[i])
    else:
        Hj = transZ(q[i])
        
    Hiden = Hiden * Tx * Rx * Tz * Rz * Hj
    
H0e = Hiden * Hne
H0e

Matrix([
[-sin(q1)*cos(q2),  sin(q1)*sin(q2)*sin(q3) - cos(q1)*cos(q3), sin(q1)*sin(q2)*cos(q3) + sin(q3)*cos(q1), -l1*sin(q1)*cos(q2) - l2*sin(q1)*cos(q2) - q4*sin(q1)*cos(q2)],
[ cos(q1)*cos(q2), -sin(q1)*cos(q3) - sin(q2)*sin(q3)*cos(q1), sin(q1)*sin(q3) - sin(q2)*cos(q1)*cos(q3),  l1*cos(q1)*cos(q2) + l2*cos(q1)*cos(q2) + q4*cos(q1)*cos(q2)],
[         sin(q2),                            sin(q3)*cos(q2),                           cos(q2)*cos(q3),                     l0 + l1*sin(q2) + l2*sin(q2) + q4*sin(q2)],
[               0,                                          0,                                         0,                                                             1]])

In [75]:
R = sp.simplify(H0e[0:3, 0:3])
P = sp.simplify(H0e[0:3, 3])

display(Math(r'\mathbf{R} = ' + sp.latex(R)))
display(Math(r'\mathbf{P} = ' + sp.latex(P)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [55]:
sp.pycode(R)

'ImmutableDenseMatrix([[-math.sin(q1)*math.cos(q2), math.sin(q1)*math.sin(q2)*math.sin(q3) - math.cos(q1)*math.cos(q3), math.sin(q1)*math.sin(q2)*math.cos(q3) + math.sin(q3)*math.cos(q1)], [math.cos(q1)*math.cos(q2), -math.sin(q1)*math.cos(q3) - math.sin(q2)*math.sin(q3)*math.cos(q1), math.sin(q1)*math.sin(q3) - math.sin(q2)*math.cos(q1)*math.cos(q3)], [math.sin(q2), math.sin(q3)*math.cos(q2), math.cos(q2)*math.cos(q3)]])'

In [56]:
sp.pycode(P)

'ImmutableDenseMatrix([[(-l1 - l2 - q4)*math.sin(q1)*math.cos(q2)], [(l1 + l2 + q4)*math.cos(q1)*math.cos(q2)], [l0 + l1*math.sin(q2) + l2*math.sin(q2) + q4*math.sin(q2)]])'

In [ ]:
[[-np.sin(q1)*np.cos(q2), np.sin(q1)*np.sin(q2)*np.sin(q3) - np.cos(q1)*np.cos(q3), np.sin(q1)*np.sin(q2)*np.cos(q3) + np.sin(q3)*np.cos(q1)],
 [np.cos(q1)*np.cos(q2), -np.sin(q1)*np.cos(q3) - np.sin(q2)*np.sin(q3)*np.cos(q1), np.sin(q1)*np.sin(q3) - np.sin(q2)*np.cos(q1)*np.cos(q3)],
 [np.sin(q2), np.sin(q3)*np.cos(q2), np.cos(q2)*np.cos(q3)]]

[[(-l1 - l2 - q4)*np.sin(q1)*np.cos(q2)],
 [(l1 + l2 + q4)*np.cos(q1)*np.cos(q2)],
 [l0 + l1*np.sin(q2) + l2*np.sin(q2) + q4*np.sin(q2)]]

In [66]:
testArr = np.array([[1],[2],[3]])
testArr[1, 0]

2

In [70]:
degree = np.pi/180.0
[q1,q2,q3,q4] = [30.0,60.0,90.0,0.5]
[q1, q2, q3] = [i*degree for i in [q1, q2, q3]]

print(q1, q2, q3, q4)

0.5235987755982988 1.0471975511965976 1.5707963267948966 0.5


In [72]:
# testArr2 = np.array('d', [[1],[2],[3]])

TypeError: data type not understood